In [13]:
import radiant as rad
import numpy as np
import matplotlib.pyplot as plt


# Problem Parameters
a = 0.
b = 1.


def exact(*x):
    return np.prod(np.cos(np.asarray(x) * 2 * np.pi / (b - a)), axis=0)


def f(*x):
    return (len(x) * (2 * np.pi / (b - a)) ** 2 + 1) * exact(*x)

In [ ]:
# Parameters
d = 2
k = 2
deltas = np.arange(0.2, 2.1, 0.1)
N = 10

# Computed Parameters
ranges = ((a, b),) * d
xc = rad.gridn(ranges, N, flat=True)

# Integration
integrator = rad.integrate.MeanIntegrator(ranges, 250)
error_integrator = rad.integrate.QuadIntegrator(ranges)

# Data collection
results = np.zeros((np.size(deltas), 3))

for i, delta in enumerate(deltas):
    # Solve for approximate solution
    phi = rad.Wendland(d, k, delta, xc)
    solver = rad.solve.GalerkinSolver(phi, integrator, 'helmholtz')
    approx = solver.solve(f)
    error = rad.error(exact, approx, error_integrator, relative=False)
    cond = solver.cond()
    bw = np.max(solver.bandwidth())
    
    results[i,0] = error
    results[i, 1] = cond
    results[i, 2] = bw
    
    print("Delta:", delta)
    print("  L2 Relative Error:", error)
    print("  Condition Number:", cond)
    print("  Bandwidth:", bw)

Delta: 0.2
  L2 Relative Error: 0.0796543633990602
  Condition Number: 57.05282686910574
  Bandwidth: 31
Delta: 0.30000000000000004
  L2 Relative Error: 0.052796258856341045
  Condition Number: 766.2646810886297
  Bandwidth: 52
Delta: 0.4000000000000001
  L2 Relative Error: 0.016786897157894
  Condition Number: 1213.2758366036815
  Bandwidth: 71
Delta: 0.5000000000000001
  L2 Relative Error: 0.017053985487117115
  Condition Number: 16176.50550792836
  Bandwidth: 84
Delta: 0.6000000000000001
  L2 Relative Error: 0.017470347963702122
  Condition Number: 117277.28510865156
  Bandwidth: 95
Delta: 0.7000000000000002
  L2 Relative Error: 0.017578189182666796
  Condition Number: 745433.939389908
  Bandwidth: 98
Delta: 0.8000000000000003
  L2 Relative Error: 0.017581957015986773
  Condition Number: 3176997.0706298114
  Bandwidth: 99
Delta: 0.9000000000000001
  L2 Relative Error: 0.017564875189891166
  Condition Number: 11701030.586431105
  Bandwidth: 99
Delta: 1.0000000000000002
  L2 Relative 

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12,3))

axs[0].set_xlabel(r"$\delta$")
axs[0].set_ylabel(r"$\| u - s\|_{L^{2}}$")
axs[0].set_yscale('log')
axs[0].plot(deltas, results[:, 0], 'o-')

axs[1].set_xlabel(r"$\delta$")
axs[1].set_ylabel(r"$\kappa_{2}\left(A_{L}\right)$")
axs[1].set_yscale('log')
axs[1].plot(deltas, results[:, 1], 'o-')

axs[2].set_xlabel(r"$\delta$")
axs[2].set_ylabel(r"$A_{L}$ Bandwidth")
axs[2].plot(deltas, results[:, 2], 'o-')

plt.subplots_adjust(wspace=0.5)
plt.savefig("figures/c4-galerkin-helmholtz-deltas.pdf", bbox_inches='tight')